In [1]:
# Imports
import pandas as pd
import numpy as np
import polars as pl
import os
import awpy
from demoparser2 import DemoParser
from awpy import Demo
from pathlib import Path
from awpy.stats import adr
from awpy.stats import kast
from awpy.stats import rating
from awpy.stats import calculate_trades
from supabase import create_client, Client
from dotenv import load_dotenv
from requests import post, get

In [2]:
folder_path = r'C:\Users\bayli\Documents\CS Demos\test_demos'
file_test = r'C:\Users\bayli\Documents\CS Demos\test_demos\natus-vincere-vs-faze-m1-inferno.dem'

# Creating DataFrames
df_flashes = pd.DataFrame()
df_he = pd.DataFrame()
df_infernos = pd.DataFrame()
df_smoke = pd.DataFrame()
df_kills = pd.DataFrame()
df_rounds = pd.DataFrame()
df_all_first_kills = pd.DataFrame()
df_adr = pd.DataFrame()
df_kast = pd.DataFrame()
df_util_dmg = pd.DataFrame()
team_rounds_won = pd.DataFrame()
players_id = pd.DataFrame()
df_matches = pd.DataFrame(columns=['event_id','match_name'])
i = 1
current_schema = "staging"
event_id = 3

In [3]:
def add_round_winners(ticks_df, rounds_df):
    ticks_df = ticks_df.to_pandas()
    rounds_df = rounds_df.to_pandas()

    # Makes sure the columns exists
    rounds_df['ct_team_clan_name'] = None
    rounds_df['t_team_clan_name'] = None
    rounds_df['winner_clan_name'] = None
    rounds_df['ct_team_current_equip_value'] = None
    rounds_df['t_team_current_equip_value'] = None
    rounds_df['ct_losing_streak'] = None
    rounds_df['t_losing_streak'] = None

    for idx, row in rounds_df.iterrows():
        freeze_end_tick = row['freeze_end']
        winner = row['winner']

        # Takes all corresponding entries
        first_tick_df = ticks_df[ticks_df['tick'] == freeze_end_tick]

        # Takes the name for every team
        try:
            CT_team = first_tick_df[first_tick_df['side'] == 'ct']['team_clan_name'].iloc[0]
        except IndexError:
            CT_team = None
        
        try:
            T_team = first_tick_df[first_tick_df['side'] == 't']['team_clan_name'].iloc[0]
        except IndexError:
            T_team = None

        # Takes the current equip value for every team
        try:
            CT_team_current_equip_value = first_tick_df[first_tick_df['side'] == 'ct']['current_equip_value'].sum()
        except KeyError:
            CT_team_current_equip_value = None

        try:
            T_team_current_equip_value = first_tick_df[first_tick_df['side'] == 't']['current_equip_value'].sum()
        except KeyError:
            T_team_current_equip_value = None

        # Determines the winner team name
        if winner == 'ct':
            winner_clan = CT_team
        elif winner in ['t', 'TERRORIST']:
            winner_clan = T_team
        else:
            winner_clan = None
            print(f"[!] Round {idx} - winner error: '{winner}'")
            
        # Fill Columns in the DataFrame
        rounds_df.at[idx, 'ct_team_clan_name'] = CT_team
        rounds_df.at[idx, 't_team_clan_name'] = T_team
        rounds_df.at[idx, 'winner_clan_name'] = winner_clan
        rounds_df.at[idx, 'ct_team_current_equip_value'] = CT_team_current_equip_value
        rounds_df.at[idx, 't_team_current_equip_value'] = T_team_current_equip_value


    return rounds_df

def add_losing_streaks(df: pd.DataFrame) -> pd.DataFrame:
    ct_losing_streak = []
    t_losing_streak = []

    ct_streak = 0
    t_streak = 0

    for _, row in df.iterrows():
        ct_team = row['ct_team_clan_name']
        t_team = row['t_team_clan_name']
        winner = row['winner_clan_name']
        
        if winner == ct_team:
            ct_streak = 0
            t_streak += 1
        else:  # winner == t_team
            t_streak = 0
            ct_streak += 1

        ct_losing_streak.append(ct_streak)
        t_losing_streak.append(t_streak)

    df['ct_losing_streak'] = ct_losing_streak
    df['t_losing_streak'] = t_losing_streak

    return df

def add_buy_type(row):

    if row['round_num'] in [1, 13]:
        return "Pistol", "Pistol"

    if row['ct_team_current_equip_value'] < 5000:
        ct_buy_type = "Full Eco"
    elif 5000 <= row['ct_team_current_equip_value'] < 10000:
        ct_buy_type = "Semi-Eco"
    elif 10000 <= row['ct_team_current_equip_value'] < 20000:
        ct_buy_type = "Semi-Buy"
    elif row['ct_team_current_equip_value'] >= 20000:
        ct_buy_type = "Full Buy"
    else:
        ct_buy_type = "Unknown"

    if row['t_team_current_equip_value'] < 5000:
        t_buy_type = "Full Eco"
    elif 5000 <= row['t_team_current_equip_value'] < 10000:
        t_buy_type = "Semi-Eco"
    elif 10000 <= row['t_team_current_equip_value'] < 20000:
        t_buy_type = "Semi-Buy"
    elif row['t_team_current_equip_value'] >= 20000:
        t_buy_type = "Full Buy"
    else:
        t_buy_type = "Unknown"

    return ct_buy_type, t_buy_type

def calculate_advantage_5v4(rounds_df, first_kills_df):

    # Makes sure the columns exists
    rounds_df['advantage_5v4'] = None

    # Checks what team got the first kill
    for idx, row in rounds_df.iterrows():
        round_num = row['round_num']

        # Filters the first kills DataFrame for the current round
        first_kill = first_kills_df[first_kills_df['round_num'] == round_num]

        if not first_kill.empty:
            # Gets the team that made the first kill
            killer_team = first_kill.iloc[0]['attacker_side']

            # Defines the advantage based on the killer team
            if killer_team == 'ct':
                rounds_df.at[idx, 'advantage_5v4'] = 'ct'
            elif killer_team == 't':
                rounds_df.at[idx, 'advantage_5v4'] = 't'

    return rounds_df

def insert_table(df, current_schema, table_name, conflict_cols):
    for row in df.to_dict(orient="records"):
        supabase.schema(current_schema).table(table_name).upsert(row, on_conflict=conflict_cols).execute()

def insert_or_update_player_history(players_df):
    for _, row in players_df.iterrows():
        steam_id = row["steam_id"]
        team_id = row["team_id"]
        
        # Finds out if the players have changed teams
        player_history_data = {
            "steam_id": steam_id,
            "team_id": team_id
        }
        # Updates the player history table with the new data
        supabase.table("player_history").upsert(
            player_history_data, 
            on_conflict=["steam_id", "team_id"]
        ).execute()

def rounds_correction(df: pl.DataFrame) -> pl.DataFrame:
    freeze_end_is_null = df.select(pl.col("freeze_end").first().is_null()).item()

    if freeze_end_is_null:
        df = df.with_columns(
            (pl.col("round_num") - 1).alias("round_num")
        )
        
    return df.filter(pl.col("freeze_end").is_not_null())

def fetch_all_rows(current_schema, table_name, page_size=1000):
    offset = 0
    all_data = []

    while True:
        response = supabase.schema(current_schema).table(table_name).select("*").range(offset, offset + page_size - 1).execute()
        data = response.data
        if not data:
            break
        all_data.extend(data)
        offset += page_size

    return all_data

In [ ]:
# Full Loop
for file_name in os.listdir(folder_path):
    if file_name.endswith('.dem'):

        file_path = os.path.join(folder_path, file_name)
        dem = Demo(file_path)
        dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "ct_losing_streak", "t_losing_streak"])

        # Gets all the Players' steam_ids
        this_file_players_id = dem.events.get('player_spawn')
        this_file_players_id = this_file_players_id.with_columns(
            this_file_players_id['user_steamid'].cast(pl.Utf8)
        )
        this_file_players_id = this_file_players_id.to_pandas()
        this_file_players_id = this_file_players_id[['user_steamid', 'user_name']].drop_duplicates()
        players_id = pd.concat([players_id, this_file_players_id], ignore_index=True)
        players_id = players_id[['user_steamid', 'user_name']].drop_duplicates()

        # Grenades Data
        # Makes that the data frame is not empty and that the columns are in the right format
        this_file_flashes = dem.events.get('flashbang_detonate', pl.DataFrame())
        if this_file_flashes is not None and len(this_file_flashes) > 0:
            this_file_flashes = this_file_flashes.with_columns(
                this_file_flashes['user_steamid'].cast(pl.Utf8)
            )
        this_file_he = dem.events.get('hegrenade_detonate', pl.DataFrame())
        if this_file_he is not None and len(this_file_he) > 0:
            this_file_he = this_file_he.with_columns(
                this_file_he['user_steamid'].cast(pl.Utf8)
            )
        this_file_infernos = dem.events.get('inferno_startburn', pl.DataFrame())
        if this_file_infernos is not None and len(this_file_infernos) > 0:
            this_file_infernos = this_file_infernos.with_columns(
                this_file_infernos['user_steamid'].cast(pl.Utf8)
            )
        this_file_smoke = dem.events.get('smokegrenade_detonate', pl.DataFrame())
        if this_file_smoke is not None and len(this_file_smoke) > 0:
            this_file_smoke = this_file_smoke.with_columns(
                this_file_smoke['user_steamid'].cast(pl.Utf8)
            )
        this_file_util_dmg = dem.events.get('player_hurt', pl.DataFrame())
        if this_file_util_dmg is not None and len(this_file_util_dmg) > 0:
            this_file_util_dmg = this_file_util_dmg.with_columns(
                this_file_util_dmg['attacker_steamid'].cast(pl.Utf8)
            )
        util_dmg = this_file_util_dmg.filter(
            (this_file_util_dmg["weapon"] == "hegrenade") |
            (this_file_util_dmg["weapon"] == "molotov")   |
            (this_file_util_dmg["weapon"] == "inferno")
        )
        # Makes sure that the data frames are not empty, converts them to pandas and appends them to the main data frame
        if this_file_flashes is not None and len(this_file_flashes) > 0:
            df_flashes = pd.concat([df_flashes, this_file_flashes.to_pandas()], ignore_index=True)
        if this_file_he is not None and len(this_file_he) > 0:
            df_he = pd.concat([df_he, this_file_he.to_pandas()], ignore_index=True)
        if this_file_infernos is not None and len(this_file_infernos) > 0:
            df_infernos = pd.concat([df_infernos, this_file_infernos.to_pandas()], ignore_index=True)
        if this_file_smoke is not None and len(this_file_smoke) > 0:
            df_smoke = pd.concat([df_smoke, this_file_smoke.to_pandas()], ignore_index=True)
        if this_file_util_dmg is not None and len(this_file_util_dmg) > 0:
            df_util_dmg = pd.concat([df_util_dmg, this_file_util_dmg.to_pandas()], ignore_index=True)

        # Opening Kills Data
        this_file_df_kills = awpy.stats.calculate_trades(demo=dem)
        this_file_df_kills = this_file_df_kills.with_columns(
            this_file_df_kills['attacker_steamid'].cast(pl.Utf8),
            this_file_df_kills['assister_steamid'].cast(pl.Utf8),
            this_file_df_kills['victim_steamid'].cast(pl.Utf8)
        )
        this_file_df_kills = this_file_df_kills.to_pandas()
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)

        # Creates Match Table
        folder_name = os.path.basename(os.path.dirname(file_path))
        file_name = file_name.replace(f"{folder_name}_", "")
        df_matches = pd.concat([df_matches, pd.DataFrame({'match_name': [file_name], 'event_id': [event_id]})], ignore_index=True)

        # Rounds Data
        this_file_df_ticks = dem.ticks
        this_file_df_rounds = dem.rounds
        this_file_df_rounds = rounds_correction(this_file_df_rounds)
        this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)
        this_file_df_rounds = add_losing_streaks(this_file_df_rounds)
        this_file_df_rounds[['ct_buy_type', 't_buy_type']] = this_file_df_rounds.apply(add_buy_type, axis=1, result_type='expand')
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)   

        this_file_df_rounds = calculate_advantage_5v4(this_file_df_rounds, df_all_first_kills)
        this_file_df_rounds['match_name'] = file_name
        df_rounds = pd.concat([df_rounds, this_file_df_rounds], ignore_index=True)
        df_rounds['event_id'] = event_id
        # Creates rounds won columns
        this_file_team_rounds_won = this_file_df_rounds.groupby('winner_clan_name').agg(
            total_rounds_won=('winner_clan_name', 'size'),
            t_rounds_won=('winner', lambda x: (x == 'ct').sum()),
            ct_rounds_won=('winner', lambda x: (x == 't').sum())
        ).reset_index()
        this_file_team_rounds_won.columns = ['team_clan_name', 'total_rounds_won','t_rounds_won', 'ct_rounds_won']
        team_rounds_won = pd.concat([team_rounds_won,this_file_team_rounds_won], ignore_index=True)
        df_kills = pd.concat([df_kills,this_file_df_kills], ignore_index=True)

        # ADR Data
        this_file_adr = awpy.stats.adr(demo=dem)
        this_file_adr = this_file_adr.with_columns(
            this_file_adr['steamid'].cast(pl.Utf8)
        )
        this_file_adr = this_file_adr.to_pandas()
        this_file_adr = this_file_adr.drop(['adr', 'name'], axis=1)
        df_adr = pd.concat([df_adr, this_file_adr], ignore_index=True)
        df_adr = df_adr.groupby(['steamid','side'], as_index=False).sum()
        df_adr = df_adr[df_adr['side'] != 'all']


        # KAST Data
        this_file_kast = awpy.stats.kast(demo=dem)
        this_file_kast = this_file_kast.with_columns(
            this_file_kast['steamid'].cast(pl.Utf8)
        )
        this_file_kast = this_file_kast.to_pandas()
        this_file_kast = this_file_kast.drop(['kast', 'name'], axis=1)
        df_kast = pd.concat([df_kast, this_file_kast], ignore_index=True)
        df_kast = df_kast.groupby(['steamid','side'], as_index=False).sum()
        df_kast = df_kast[df_kast['side'] != 'all']

        print(f"{i}: Processed {file_name}")
        i = i + 1

In [4]:
dem = Demo(file_test)
dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "round_num", "is_alive"])

In [59]:
pl.Config.set_tbl_rows(10)

polars.config.Config

In [70]:
dem.rounds

round_num,start,freeze_end,end,official_end,winner,reason,bomb_plant,bomb_site
u32,i32,i32,i32,i32,str,str,i64,str
1,540,6363,12448,12896,"""t""","""bomb_exploded""",9824,"""bombsite_b"""
2,12896,14176,23643,24091,"""t""","""bomb_exploded""",21019,"""bombsite_b"""
3,24091,25371,29749,30197,"""ct""","""t_killed""",null,"""not_planted"""
4,30197,31477,38837,39285,"""ct""","""time_ran_out""",null,"""not_planted"""
5,39285,40565,47343,47791,"""ct""","""t_killed""",null,"""not_planted"""
6,47791,49071,58805,59253,"""t""","""bomb_exploded""",56181,"""bombsite_b"""
7,59253,63458,70056,70504,"""ct""","""bomb_defused""",67585,"""bombsite_b"""
8,70504,71784,78962,79410,"""t""","""ct_killed""",77678,"""bombsite_b"""
9,79410,80690,88050,88498,"""ct""","""time_ran_out""",null,"""not_planted"""


In [ ]:
dem.kills

In [61]:
ticks = dem.ticks
ticks.tail(5)

total_rounds_played,health,place,current_equip_value,side,team_clan_name,X,Y,Z,is_alive,tick,steamid,name,round_num
i32,i32,str,u32,str,str,f32,f32,f32,bool,i32,u64,str,u32
18,0,"""BombsiteA""",5000,"""t""","""Natus Vincere""",2241.293457,1012.108093,160.850372,false,183337,76561198013243326,"""Aleksib""",19
18,0,"""Apartments""",5600,"""t""","""Natus Vincere""",1941.304077,-188.675842,260.03125,false,183337,76561198246607476,"""b1t""",19
18,71,"""BombsiteA""",4600,"""t""","""Natus Vincere""",1966.831665,137.574722,142.664764,true,183337,76561198176878303,"""jL""",19
18,0,"""TopofMid""",4250,"""ct""","""FaZe Clan""",1355.174561,220.966873,133.579147,false,183337,76561197989430253,"""karrigan""",19
18,0,"""TopofMid""",5100,"""t""","""Natus Vincere""",1420.387695,1171.80603,177.645782,false,183337,76561198050250233,"""iM""",19


In [73]:
clutches_data = []

all_players = dem.ticks.select(["steamid", "name"]).unique()
player_name_map = {row["steamid"]: row["name"] for row in all_players.iter_rows(named=True)}

for round_info in dem.rounds.iter_rows(named=True):
    round_num = round_info['round_num']
    round_start_tick = round_info['start']
    round_end_tick = round_info['end']
    winning_team = round_info['winner'] # Ex: 'CT' ou 'T'

    # Filtrar ticks e kills para a rodada atual
    round_ticks_df = dem.ticks.filter(
        (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
    )
    round_kills_df = dem.kills.filter(
        (pl.col("round_num") == round_num) &
        (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
    ).sort("tick")

    # Preparar os dados de jogadores vivos por tick para a rodada
    alive_counts_per_tick = round_ticks_df.group_by("tick").agg(
        (pl.when(pl.col("side") == "ct").then(pl.col("is_alive")).otherwise(0)).sum().alias("ct_alive"),
        (pl.when(pl.col("side") == "t").then(pl.col("is_alive")).otherwise(0)).sum().alias("t_alive")
    )
    
    players_alive_at_tick = round_ticks_df.group_by("tick").agg(
        pl.struct(["steamid", "is_alive", "side"]).alias("players_state")
    )
    
    # Juntar os dados e garantir a ordenação por tick
    merged_round_data = alive_counts_per_tick.join(
        players_alive_at_tick, on="tick", how="left"
    ).sort("tick")


    # Variáveis de controle do estado do clutch
    clutch_active = False
    clutcher_steamid = None
    clutcher_team_side = None
    clutch_start_tick = None
    opponents_at_start = 0

    for i, tick_data in enumerate(merged_round_data.iter_rows(named=True)):
        current_tick = tick_data['tick']
        ct_alive = tick_data['ct_alive']
        t_alive = tick = tick_data['t_alive']
        players_state = tick_data['players_state']
        
        is_clutch_condition_met = (ct_alive == 1 and t_alive >= 2) or \
                                    (t_alive == 1 and ct_alive >= 2)

        clutcher_is_alive_this_tick = False
        if clutch_active and clutcher_steamid:
            for p_state in players_state:
                if p_state['steamid'] == clutcher_steamid and p_state['is_alive']:
                    clutcher_is_alive_this_tick = True
                    break

        if is_clutch_condition_met:
            if not clutch_active: # Início de um NOVO clutch
                clutch_active = True
                clutch_start_tick = current_tick
                
                if ct_alive == 1:
                    clutcher_team_side = 'ct'
                    opponents_at_start = t_alive # Captura oponentes no início
                else: # t_alive == 1
                    clutcher_team_side = 't'
                    opponents_at_start = ct_alive # Captura oponentes no início
                
                for p_state in players_state:
                    if p_state['is_alive'] and p_state['side'] == clutcher_team_side:
                        clutcher_steamid = p_state['steamid']
                        break
        
        else: # A condição de clutch (1vX, X>=2) NÃO é mais atendida neste tick
            if clutch_active: # Mas havia um clutch ativo no tick anterior
                # O clutch termina se o clutcher morreu
                if not clutcher_is_alive_this_tick:
                    clutch_end_tick = current_tick # Clutch termina com a morte do clutcher
                    clutch_won = False # Clutch perdido por morte
                    
                    final_clutch_kills = 0
                    if clutcher_steamid:
                        final_clutch_kills = round_kills_df.filter(
                            (pl.col("tick") >= clutch_start_tick) &
                            (pl.col("tick") <= clutch_end_tick) &
                            (pl.col("attacker_steamid") == clutcher_steamid)
                        ).shape[0]

                    # Determina se o time do clutcher venceu a rodada
                    # Convertendo winning_team para minúsculas para comparação consistente
                    round_team_won = (clutcher_team_side == winning_team.lower()) 

                    clutches_data.append({
                        "round_num": round_num,
                        "clutcher_steamid": clutcher_steamid,
                        "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
                        "clutcher_team_side": clutcher_team_side,
                        "opponents_at_start": opponents_at_start,
                        "clutch_start_tick": clutch_start_tick,
                        "clutch_end_tick": clutch_end_tick,
                        "clutch_kills": final_clutch_kills,
                        "clutch_won": clutch_won,
                        "round_won": round_team_won # Nova coluna
                    })
                    
                    clutch_active = False
                    clutcher_steamid = None
                    clutcher_team_side = None
                    clutch_start_tick = None
                    opponents_at_start = 0

    # --- Lógica de encerramento no final da rodada ---
    if clutch_active:
        clutch_end_tick = round_end_tick 
        
        final_clutch_kills = 0
        if clutcher_steamid:
            final_clutch_kills = round_kills_df.filter(
                (pl.col("tick") >= clutch_start_tick) &
                (pl.col("tick") <= clutch_end_tick) &
                (pl.col("attacker_steamid") == clutcher_steamid)
            ).shape[0]

        clutch_won = False
        
        # Determina se o time do clutcher venceu a rodada
        # Convertendo winning_team para minúsculas para comparação consistente
        round_team_won = (clutcher_team_side == winning_team.lower())

        # Se o clutcher chegou até o final da rodada com o clutch ativo,
        # ele venceu o clutch se o time dele venceu a rodada.
        if round_team_won:
            clutch_won = True
        
        clutches_data.append({
            "round_num": round_num,
            "clutcher_steamid": clutcher_steamid,
            "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
            "clutcher_team_side": clutcher_team_side,
            "opponents_at_start": opponents_at_start,
            "clutch_start_tick": clutch_start_tick,
            "clutch_end_tick": clutch_end_tick,
            "clutch_kills": final_clutch_kills,
            "clutch_won": clutch_won,
            "round_won": round_team_won # Nova coluna
        })
        
        clutch_active = False
        clutcher_steamid = None
        clutcher_team_side = None
        clutch_start_tick = None
        opponents_at_start = 0

clutches_df = pl.DataFrame(clutches_data)

In [76]:
clutches_data = []

all_players = dem.ticks.select(["steamid", "name"]).unique()
player_name_map = {row["steamid"]: row["name"] for row in all_players.iter_rows(named=True)}

for round_info in dem.rounds.iter_rows(named=True):
    round_num = round_info['round_num']
    round_start_tick = round_info['start']
    round_end_tick = round_info['end']
    winning_team = round_info['winner'] # Ex: 'CT' ou 'T'

    # Filtrar ticks e kills para a rodada atual
    round_ticks_df = dem.ticks.filter(
        (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
    )
    round_kills_df = dem.kills.filter(
        (pl.col("round_num") == round_num) &
        (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
    ).sort("tick")

    # Preparar os dados de jogadores vivos por tick para a rodada
    alive_counts_per_tick = round_ticks_df.group_by("tick").agg(
        (pl.when(pl.col("side") == "ct").then(pl.col("is_alive")).otherwise(0)).sum().alias("ct_alive"),
        (pl.when(pl.col("side") == "t").then(pl.col("is_alive")).otherwise(0)).sum().alias("t_alive")
    )
    
    players_alive_at_tick = round_ticks_df.group_by("tick").agg(
        pl.struct(["steamid", "is_alive", "side"]).alias("players_state")
    )
    
    # Juntar os dados e garantir a ordenação por tick
    merged_round_data = alive_counts_per_tick.join(
        players_alive_at_tick, on="tick", how="left"
    ).sort("tick")


    # Variáveis de controle do estado do clutch
    clutch_active = False
    clutcher_steamid = None
    clutcher_team_side = None
    clutch_start_tick = None
    opponents_at_start = 0

    for i, tick_data in enumerate(merged_round_data.iter_rows(named=True)):
        current_tick = tick_data['tick']
        ct_alive = tick_data['ct_alive']
        t_alive = tick_data['t_alive']
        players_state = tick_data['players_state']
        
        is_clutch_condition_met = (ct_alive == 1 and t_alive >= 2) or \
                                    (t_alive == 1 and ct_alive >= 2)

        clutcher_is_alive_this_tick = False
        if clutch_active and clutcher_steamid:
            for p_state in players_state:
                if p_state['steamid'] == clutcher_steamid and p_state['is_alive']:
                    clutcher_is_alive_this_tick = True
                    break

        if is_clutch_condition_met:
            if not clutch_active: # Início de um NOVO clutch
                clutch_active = True
                clutch_start_tick = current_tick
                
                if ct_alive == 1:
                    clutcher_team_side = 'ct'
                    opponents_at_start = t_alive # Captura oponentes no início
                else: # t_alive == 1
                    clutcher_team_side = 't'
                    opponents_at_start = ct_alive # Captura oponentes no início
                
                for p_state in players_state:
                    if p_state['is_alive'] and p_state['side'] == clutcher_team_side:
                        clutcher_steamid = p_state['steamid']
                        break
        
        else: # A condição de clutch (1vX, X>=2) NÃO é mais atendida neste tick
            if clutch_active: # Mas havia um clutch ativo no tick anterior
                # O clutch termina se o clutcher morreu
                if not clutcher_is_alive_this_tick:
                    clutch_end_tick = current_tick # Clutch termina com a morte do clutcher
                    clutch_won = False # Clutch perdido por morte
                    
                    final_clutch_kills = 0
                    if clutcher_steamid:
                        final_clutch_kills = round_kills_df.filter(
                            (pl.col("tick") >= clutch_start_tick) &
                            (pl.col("tick") <= clutch_end_tick) &
                            (pl.col("attacker_steamid") == clutcher_steamid)
                        ).shape[0]

                    # Determina se o time do clutcher venceu a rodada
                    round_team_won = (clutcher_team_side == winning_team.lower()) 

                    # NOVO: Clutcher não sobreviveu ao round se ele morreu para encerrar o clutch
                    clutcher_survived_round = False

                    clutches_data.append({
                        "round_num": round_num,
                        "clutcher_steamid": clutcher_steamid,
                        "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
                        "clutcher_team_side": clutcher_team_side,
                        "opponents_at_start": opponents_at_start,
                        "clutch_start_tick": clutch_start_tick,
                        "clutch_end_tick": clutch_end_tick,
                        "clutch_kills": final_clutch_kills,
                        "clutch_won": clutch_won,
                        "round_won": round_team_won,
                        "clutcher_survived_round": clutcher_survived_round # Nova coluna
                    })
                    
                    clutch_active = False
                    clutcher_steamid = None
                    clutcher_team_side = None
                    clutch_start_tick = None
                    opponents_at_start = 0

    # --- Lógica de encerramento no final da rodada ---
    if clutch_active:
        clutch_end_tick = round_end_tick 
        
        final_clutch_kills = 0
        if clutcher_steamid:
            final_clutch_kills = round_kills_df.filter(
                (pl.col("tick") >= clutch_start_tick) &
                (pl.col("tick") <= clutch_end_tick) &
                (pl.col("attacker_steamid") == clutcher_steamid)
            ).shape[0]

        clutch_won = False
        
        # Determina se o time do clutcher venceu a rodada
        round_team_won = (clutcher_team_side == winning_team.lower())

        # Se o clutcher chegou até o final da rodada com o clutch ativo,
        # ele venceu o clutch se o time dele venceu a rodada.
        if round_team_won:
            clutch_won = True
        
        # NOVO: Determinar se o clutcher sobreviveu ao round
        clutcher_survived_round = False
        if clutcher_steamid:
            # Busca o último estado de vida do clutcher até o round_end_tick
            last_known_state = round_ticks_df.filter(
                (pl.col("tick") <= round_end_tick) & 
                (pl.col("steamid") == clutcher_steamid)
            ).sort("tick", descending=True).select("is_alive").head(1) 
            
            if last_known_state.height == 1:
                clutcher_survived_round = last_known_state.item()
        
        clutches_data.append({
            "round_num": round_num,
            "clutcher_steamid": clutcher_steamid,
            "clutcher_name": player_name_map.get(clutcher_steamid, "Unknown"),
            "clutcher_team_side": clutcher_team_side,
            "opponents_at_start": opponents_at_start,
            "clutch_start_tick": clutch_start_tick,
            "clutch_end_tick": clutch_end_tick,
            "clutch_kills": final_clutch_kills,
            "clutch_won": clutch_won,
            "round_won": round_team_won,
            "clutcher_survived_round": clutcher_survived_round # Nova coluna
        })
        
        clutch_active = False
        clutcher_steamid = None
        clutcher_team_side = None
        clutch_start_tick = None
        opponents_at_start = 0

clutches_df = pl.DataFrame(clutches_data)

In [77]:
clutches_df

round_num,clutcher_steamid,clutcher_name,clutcher_team_side,opponents_at_start,clutch_start_tick,clutch_end_tick,clutch_kills,clutch_won,round_won,clutcher_survived_round
i64,i64,str,str,i64,i64,i64,i64,bool,bool,bool
1,76561199063068840,"""w0nderful""","""ct""",2,12017,12448,0,false,false,true
3,76561198068422762,"""frozen""","""t""",5,28567,29749,0,false,false,false
4,76561197991272318,"""ropz""","""t""",4,36701,38837,0,false,false,true
5,76561197989430253,"""karrigan""","""t""",5,47176,47343,1,false,false,false
6,76561198013243326,"""Aleksib""","""ct""",4,56609,58805,1,false,false,true
7,76561197997351207,"""rain""","""t""",2,69151,69524,0,false,false,false
8,76561199063068840,"""w0nderful""","""ct""",2,77268,78962,0,false,false,false
9,76561198201620490,"""broky""","""t""",5,84443,88050,2,false,false,true
10,76561197989430253,"""karrigan""","""t""",5,95545,95721,1,false,false,false
